In [1]:
import os, random, numpy as np, pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.transforms import Resize
from torch.amp import autocast, GradScaler
from torch.utils.checkpoint import checkpoint_sequential
from torch.utils.checkpoint import checkpoint

# === REPRODUCIBILITY ===
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# === PATHS (COLAB) ===
train_csv_path = "/content/drive/MyDrive/Colab Notebooks/Projects/CSVs/train.csv"
test_csv_path  = "/content/drive/MyDrive/Colab Notebooks/Projects/CSVs/test.csv"
NPY_DIR        = "/content/drive/MyDrive/Colab Notebooks/Projects/npy_segments_unimodal"
save_path      = "/content/drive/MyDrive/Colab Notebooks/Results/Unfrozen_randomseed/CUENET"
os.makedirs(save_path, exist_ok=True)

# === CONFIG ===
BATCH_SIZE = 2
MAX_FRAMES = 80
EPOCHS = 20
USE_WEIGHTED_LOSS = True
PATIENCE = 4

# === DATASET ===
class ViolenceDataset(Dataset):
    def __init__(self, csv_path, npy_dir):
        self.df = pd.read_csv(csv_path)
        self.npy_dir = npy_dir
        self.resize = Resize((224, 224))
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        frames = np.load(os.path.join(self.npy_dir, f"{row['Segment ID']}.npy"))[:MAX_FRAMES]
        frames = torch.stack([
            self.resize(torch.from_numpy(f).permute(2,0,1).float()/255.0)
            for f in frames
        ])
        return frames, torch.tensor(row['Violence label(video)'], dtype=torch.float32)

# === CUE-Net Model ===
class CUENet(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = resnet50(weights=ResNet50_Weights.DEFAULT)
        self.backbone.fc = nn.Identity()  # remove classifier
        self.temporal_gru = nn.GRU(2048, 512, batch_first=True, bidirectional=True)
        self.attention_fc = nn.Linear(1024, 1)
        self.fc_out = nn.Linear(1024, 1)
    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B*T, C, H, W)
        # Apply checkpoint to backbone forward
        feats = checkpoint(self.backbone, x, use_reentrant=False)
        feats = feats.view(B, T, -1)
        gru_out, _ = self.temporal_gru(feats)
        attn_weights = torch.softmax(self.attention_fc(gru_out), dim=1)
        weighted = torch.sum(gru_out * attn_weights, dim=1)
        return self.fc_out(weighted).squeeze(1)

# === INIT ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dataset = ViolenceDataset(train_csv_path, NPY_DIR)
test_dataset  = ViolenceDataset(test_csv_path,  NPY_DIR)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=BATCH_SIZE, shuffle=False)

pos = train_dataset.df['Violence label(video)'].sum()
neg = len(train_dataset) - pos
ratio = neg / max(pos, 1)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([ratio]).to(DEVICE)) if USE_WEIGHTED_LOSS else nn.BCEWithLogitsLoss()

model = CUENet().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=1, factor=0.5)
scaler = GradScaler()

best_f1, early_stop_counter = 0, 0

# === TRAIN ===
for epoch in range(EPOCHS):
    model.train()
    y_true, y_pred, total_loss = [], [], 0.0
    for frames, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        frames, labels = frames.to(DEVICE), labels.to(DEVICE)
        with autocast(device_type='cuda'):
            outputs = model(frames)
            loss = criterion(outputs, labels)
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
        preds = (torch.sigmoid(outputs) > 0.5).int()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
    macro_f1 = f1_score(y_true, y_pred, average='macro')
    micro_f1 = f1_score(y_true, y_pred, average='micro')
    print(f"Epoch {epoch+1} | Loss: {total_loss/len(train_loader):.4f} | Macro F1: {macro_f1:.4f} | Micro F1: {micro_f1:.4f}")
    scheduler.step(macro_f1)
    if macro_f1 > best_f1:
        best_f1 = macro_f1
        torch.save(model.state_dict(), os.path.join(save_path, "cuenet_best_20.pt"))
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= PATIENCE:
            break

# === TEST ===
model.load_state_dict(torch.load(os.path.join(save_path, "cuenet_best_20.pt")))
model.eval()
y_true, y_pred, test_losses = [], [], []
segment_ids = test_dataset.df['Segment ID'].tolist()

with torch.no_grad():
    for frames, labels in test_loader:
        frames, labels = frames.to(DEVICE), labels.to(DEVICE)
        outputs = model(frames)
        loss = criterion(outputs, labels)
        test_losses.append(loss.item())
        preds = (torch.sigmoid(outputs) > 0.5).int()
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

avg_test_loss = np.mean(test_losses)
report = classification_report(y_true, y_pred, target_names=["Non-violent","Violent"], output_dict=True, zero_division=0)
conf_matrix = confusion_matrix(y_true, y_pred)

print(f"\n[TEST] BCE Loss: {avg_test_loss:.4f}")
print(f"[TEST] Macro F1: {report['macro avg']['f1-score']:.4f}")
print(f"[TEST] Micro F1: {f1_score(y_true,y_pred,average='micro'):.4f}")
print("[TEST] Per-Class F1 Scores:")
print(f" - Non-violent F1: {report['Non-violent']['f1-score']:.4f}")
print(f" - Violent F1: {report['Violent']['f1-score']:.4f}")
print("Confusion Matrix:\n", conf_matrix)

pd.DataFrame({"Segment ID": segment_ids, "True": y_true, "Pred": y_pred}).to_csv(
    os.path.join(save_path, "cuenet_predictions_20.csv"), index=False)
pd.DataFrame(report).to_csv(os.path.join(save_path, "cuenet_test_metrics_20.csv"))


Epoch 1/20: 100%|██████████| 335/335 [15:07<00:00,  2.71s/it]


Epoch 1 | Loss: 0.7087 | Macro F1: 0.6364 | Micro F1: 0.6368


Epoch 2/20: 100%|██████████| 335/335 [07:07<00:00,  1.28s/it]


Epoch 2 | Loss: 0.5724 | Macro F1: 0.7284 | Micro F1: 0.7294


Epoch 3/20: 100%|██████████| 335/335 [07:03<00:00,  1.26s/it]


...

Epoch 18 | Loss: 0.0492 | Macro F1: 0.9909 | Micro F1: 0.9910


Epoch 19/20: 100%|██████████| 335/335 [06:37<00:00,  1.19s/it]


Epoch 19 | Loss: 0.0172 | Macro F1: 0.9954 | Micro F1: 0.9955


Epoch 20/20: 100%|██████████| 335/335 [06:36<00:00,  1.18s/it]


Epoch 20 | Loss: 0.0200 | Macro F1: 0.9924 | Micro F1: 0.9925

[TEST] BCE Loss: 1.8898
[TEST] Macro F1: 0.6642
[TEST] Micro F1: 0.6718
[TEST] Per-Class F1 Scores:
 - Non-violent F1: 0.7147
 - Violent F1: 0.6137
Confusion Matrix:
 [[134  49]
 [ 58  85]]
